# TSPTW by OR-Tools

In [5]:
import random as rd
import numpy as np
import pandas as pd
import math
import copy as copy
from ortools.constraint_solver import routing_enums_pb2 
from ortools.constraint_solver import pywrapcp 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [7]:
def create_data_model():
    data = {}
    data["locations"] = [(round(np.random.uniform(0, MAX_X),2), round(np.random.uniform(0, MAX_Y),2)) for i in range(NUM_NODES)]
    data["locations"][0] = (MAX_X / 2, MAX_Y / 2)
    data["distance_matrix"] = compute_distance_matrix(data["locations"])
    data["time_matrix"] = compute_time_matrix(data["distance_matrix"])
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data

def compute_distance_matrix(nodes):
    distance_matrix = []
    for node_i in nodes:
        distances = []
        for node_j in nodes:
            if node_i == node_j:
                distances.append(0)
            else:
                manhattan_distance = np.sum(np.abs(np.array(node_i) - np.array(node_j)))
                distances.append(round(manhattan_distance, 2))
        distance_matrix.append(distances)
    return distance_matrix

def compute_time_matrix(distance_matrix):
    time_matrix = []
    for i in distance_matrix:
        time_matrix.append(list(map(lambda x: int(x * 60), i)))
    return time_matrix

In [8]:
def main():

    manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        0,  # allow waiting time
        640 * 60,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.time_limit.seconds = 10
    search_parameters.solution_limit = 100

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_second = print_solution(data, manager, routing, solution)
        cumul_data = get_cumul_data(solution, routing, time_dimension)
        return total_second
    else:
        print('No solution.')

def get_cumul_data(solution, routing, dimension):
    cumul_data = []
    for route_nbr in range(routing.vehicles()):
        route_data = []
        index = routing.Start(route_nbr)
        dim_var = dimension.CumulVar(index)
        route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            dim_var = dimension.CumulVar(index)
            route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
            cumul_data.append(route_data)
    return cumul_data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}sec\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}s")
    print(f"Total serivce time: {(NUM_NODES - 1) * 120}s")
    return (total_time + ((NUM_NODES - 1) * 120))
    

In [9]:
result = { 
    "size": [], 
    "num_node": [], 
    "total_time": []
    }

for i in range(10, 200, 10):

    NUM_NODES = i
    MAX_X, MAX_Y = (15, 15)

    for j in range(30):

        result["size"].append((MAX_X, MAX_Y))
        result["num_node"].append(i)
        data = create_data_model()
        travel_time = main()
        serivce_time = (NUM_NODES - 1) * 120
        total_time = travel_time + serivce_time
        result["total_time"].append(round(total_time / 60, 2))
        # print('-Travel Time: %.2f min' %(travel_time / 60))
        # print('-Service Time: %.2f min' %(serivce_time / 60))
        print('-Total Time: %.2f min' %(total_time / 60))

Objective: 3439
Route for vehicle 0:
0 Time(0,0) -> 3 Time(177,177) -> 9 Time(435,435) -> 5 Time(651,651) -> 6 Time(947,947) -> 2 Time(1619,1619) -> 8 Time(1701,1701) -> 7 Time(2292,2292) -> 4 Time(2613,2613) -> 1 Time(2874,2874) -> 0 Time(3439,3439)
Time of the route: 3439sec

Total time of all routes: 3439s
Total serivce time: 1080s
-Total Time: 93.32 min
Objective: 3260
Route for vehicle 0:
0 Time(0,0) -> 9 Time(212,212) -> 4 Time(687,687) -> 5 Time(921,921) -> 3 Time(1642,1642) -> 2 Time(1784,1784) -> 7 Time(2131,2131) -> 1 Time(2390,2390) -> 6 Time(2622,2622) -> 8 Time(3036,3036) -> 0 Time(3260,3260)
Time of the route: 3260sec

Total time of all routes: 3260s
Total serivce time: 1080s
-Total Time: 90.33 min
Objective: 2438
Route for vehicle 0:
0 Time(0,0) -> 2 Time(46,46) -> 3 Time(208,208) -> 7 Time(335,335) -> 1 Time(732,732) -> 4 Time(1090,1090) -> 6 Time(1222,1222) -> 9 Time(1414,1414) -> 5 Time(1778,1778) -> 8 Time(2250,2250) -> 0 Time(2438,2438)
Time of the route: 2438sec

T

In [10]:
df_result = pd.DataFrame(result)

In [11]:
df_result

,size,num_node,total_time
0,"(15, 15)",10,93.32
1,"(15, 15)",10,90.33
2,"(15, 15)",10,76.63
3,"(15, 15)",10,94.35
4,"(15, 15)",10,87.77
...,...,...,...
565,"(15, 15)",190,961.58
566,"(15, 15)",190,965.58
567,"(15, 15)",190,955.78
568,"(15, 15)",190,956.63


In [12]:
df_result.to_csv('15x15_time.csv')

In [13]:
df_result.groupby('num_node')['total_time'].mean()

num_node
10      89.247000
20     149.045333
30     201.480000
40     254.417333
50     305.757000
60     356.253667
70     403.094667
80     451.987000
90     499.590667
100    548.569000
110    593.018667
120    641.511333
130    687.029333
140    731.723333
150    780.294333
160    825.325667
170    871.182667
180    915.634667
190    962.178667
Name: total_time, dtype: float64